<a href="https://colab.research.google.com/github/lehoangan2906/MORL/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from pyexpat import model
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import random
import tensorflow as tf
import gym 
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from collections import deque
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam, RMSprop

In [ ]:
class MultiQNetwork(keras.Model):
  """
  Xây dựng Multi Q Network dựa trên 4 Fully Connected + Các tham chiếu w
  Đầu vào:
  Concatenation of State representation and parameters of a linear preferrence function
  Đầu ra:
  Giá trị Q-network với kích thước m*size|A| (m = reward_size: số mục tiêu, A: action_space)
  """
  def __init__(self, state_size, action_size, reward_size):
    super(MultiQNetwork, self).__init__()

    self.state_size = state_size
    self.action_size = action_size
    self.reward_size = reward_size

    input_shape = state_size + reward_size
    action_space = action_size * reward_size

    self.layer1 = Dense(input_shape * 16, activation = 'relu', kernel_initializer = 'he_uniform')
    self.layer2 = Dense(input_shape * 32, activation = 'relu', kernel_initializer = 'he_uniform')
    self.layer3 = Dense(input_shape * 64, activation = 'relu', kernel_initializer = 'he_uniform')
    self.layer4 = Dense(input_shape * 32, activation = 'relu', kernel_initializer = 'he_uniform')
    self.layer5 = Dense(action_space, activation = 'relu', kernel_initializer = 'he_uniform')


  def H(self, Q, w, s_num, w_num):
    """
    Toán tử HQ
    w_num: Số lượng tham chiếu lấy mẫu ra từ D_w
    S_num: Số lượng state
    """
    mask = tf.concat([np.arange(i, s_num * w_num + i, s_num) for i in range(s_num)], axis = 0)
    test = tf.reshape(Q, [-1, self.action_size * self.reward_size])

    TT = []
    for i in range(len(mask)):
      TT.append(test[mask[i]])
    TT = tf.convert_to_tensor(TT)

    reQ = tf.reshape(TT, [-1, self.reward_size]).numpy()
    reQ_ext = np.tile(reQ, (w_num, 1))
    w_ext = np.tile(tf.expand_dims(w, 2), [1, self.action_size * w_num, 1])
    w_ext = tf.reshape(w_ext, [-1, self.reward_size])

    # Nhan wT.Q
    prod = tf.reshape(tf.reduce_sum(tf.multiply(reQ_ext, w_ext), axis = 1), [-1, self.action_size * w_num])

    # Tim chi so lon nhat
    inds = np.argmax(prod, axis = 1)
    mask = np.zeros(prod.shape)
    for i in range(len(inds)):
      mask[i, inds[i]] = 1
    mask = np.reshape(mask, [-1, 1])

    HQ = []
    for i in range(len(mask)):
      if mask[i] == 1:
        HQ.append(reQ_ext[i])
    HQ = tf.convert_to_tensor(HQ)
    return HQ


  def forward(self, state, preference, w_num =1):
    """
    Trả giá trị Q và HQ
    """
    s_num = int(preference.shape[-0]/w_num)
    x = np.concatenate((state, preference), axis = 1)

    x = tf.convert_to_tensor(np.array(x))
    x = self.layer1(X)
    x = self.layer2(X)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.layer5(X)
    q = tf.reshape(q, [q.shape[0], self.action_size, self.reward_size])
    Hq = self.H(q, preference, s_num, w_num)
    return Hq, q